![banner](./logo/banner.png)

![logo](logo/banner.png)

# Bot

Para realizar este bot vamos a utilizar una libreria que nos ayudara a realizar el control de mensajes de una manera sencilla [python-telegram-bot](https://telegram.org/botfather)

El bor que haremos tiene el proposito de decirnos el clima utilizando el sitio [accuweather](https://www.accuweather.com).

## Librerias

Las librerias que vamos a utilizar son:

- `BeautifulSoup`: Analizar el sitio para obtener la temperatura
- `requests`: Realizar la solicitud a los sitios web
- `python-telegram-bot`: Conectar y controlar el bot de Telegram

## Creando Bot

Para obtener un bot de Telegram debemos contactar al [BotFhater](https://core.telegram.org/bots/api) el cual nos genera la autorizacion de generarlo. Para ello debemos seguir los siguientes pasos:

1. Entrar a Telegram y buscar a **BotFather**
2. Darle el comando `/newbot`
3. Debemos darle un nombre al bot
4. Darle un nombre de usuario al bot, debe tener la terminacio `_bot`
5. Una vez es aprobado el nombre, te otorgan un `TOKEN`
6. TOKEN generado (no lo compartas)
7. [API de telegram](https://core.telegram.org/bots/api) es para conocere lo que se puede realizar con el bot

## Archivos creados

`TOKEN.py`

```python
TOKEN='TOKEN'
```
---

`weather.py`

```python
import requests
import ast

def get_weather():

    header = { 'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)' }
    page = requests.get('https://www.accuweather.com/en/mx/coatzacoalcos/236228/weather-forecast/236228', headers=header)

    content = page.text

    lines = content.splitlines()

    text_to_find = 'recentLocations'
    text_finded = ''

    for l in lines:
        if l.find(text_to_find) >=0:
            text_finded = l.strip()
            break

    text_split = text_finded.split(" ")
    data = text_split[-1]
    ##cleaning data
    data = data.replace("[",'').replace(']','').replace(';','')

    #print(type(data))#esto fue para ver que tipo era el objeto

    #print(data)

    data = ast.literal_eval(data)
    
    country = data['adminArea']['localizedName']
    temp = data['temp']
    city = data['localizedName']

    return {
        'city':city,
        'temp': temp,
        'country': country
    }

```
---

`bot.py`

```python
from weather import get_weather
from TOKEN import TOKEN
from telegram import Update, ForceReply, update
from telegram.ext import Updater, CommandHandler, CallbackContext


def weather(update: Update, context: CallbackContext) -> None:
    data = get_weather()
    city = data['city']
    temp = data['temp']
    country = data['country']

    message = f'En {city}, {country} hay un clima de {temp}C'
    print("respuesta", message)

    update.message.reply_text(message)


def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def main():
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater(TOKEN)

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("clima", weather))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()
```

---
Realizado por Docente: [Alejandro Leyva](https://www.alejandro-leyva.com/)